In [ ]:
!python --version

In [ ]:
import os.path
import shutil
import urllib.request as request
from contextlib import closing
from pandas import DataFrame, read_csv
import pandas as pd 

In [ ]:
# Source: https://www.data.gv.at/katalog/dataset/f3a1c18ea3a038fa13e7b3c89454ed280d16ef0c
file_url = "http://data.ooe.gv.at/files/cms/Mediendateien/OGD/ogd_abtStat/"
file_name_prefix = "stat_download_nr"
file_name_pstfix = ".xls"
file_local_location = "./"
file_save_prefix = "NRW_NOE_"
file_save_pstfix = ".tsv"

In [ ]:
def download_dataset(year, file_index):
    current_file_name = file_name_prefix + file_index + file_name_pstfix
    file_store_url = file_local_location + current_file_name
    final_URL = file_url + current_file_name
    
    if os.path.isfile(file_store_url):
        print(file_store_url)
        return file_store_url
    print(final_URL)
    with closing(request.urlopen(final_URL)) as r:
        with open(file_store_url, 'wb') as f:
            shutil.copyfileobj(r, f)
            print(final_URL)
            return file_store_url

In [ ]:
def reformat_file(current_file_url, year):
    formated_data = {}
    if int(year) < 2017:
        df = pd.read_excel(current_file_url, "Stimmen", header=2)[1:]
    else: 
        df = pd.read_excel(current_file_url, "Stimmen", header=1)[1:]
    df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
    df = df[df.columns.drop(list(df.filter(regex='Eingabe')))]
    del df["Wahlbet."]
    df.rename(columns={'Nr.':'GKZ','Name':'Gebietsname','Wahlbe-':'Wahlberechtigte','abgegeb.':'Abgegebene','ungültige':'Ungültige','gültige':'Gültige'}, inplace=True)

    #print(df.head())
    return df

In [ ]:
## main
number_of_files = {"2002":"02","2006": "06", "2008":"08","2013":"13","2017":"17","2019":"19"}
for year in number_of_files:
    current_file_url = download_dataset(year, number_of_files[year])
    corrected_df = reformat_file(current_file_url, year)
    corrected_df.to_csv(file_save_prefix + str(year) + file_save_pstfix, sep="\t")
